In [ ]:
# ---------------------------------------------------------------------------
# Google Colab Script to Create Label Directory Structure for YOLOv8
# ---------------------------------------------------------------------------
# This script will:
# 1. Mount your Google Drive.
# 2. Define the path to your dataset (e.g., 'bisindo_dataset_split').
# 3. Create a 'labels' directory inside your dataset root.
# 4. Replicate the directory structure from your 'train', 'val',
#    and 'test' image folders into the 'labels' folder.
#
# IMPORTANT:
# - This script ONLY creates the directory structure.
# - You still need to populate these directories with your actual
#   '.txt' label files, ensuring each image has a corresponding
#   label file with the same name (e.g., image1.jpg -> image1.txt).
# ---------------------------------------------------------------------------

import os
from google.colab import drive # For Google Colab environments

# --- 1. Mount Google Drive ---
# This will prompt you for authorization if not already mounted.
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    # If drive mount fails, it's critical, so we might stop here.
    raise SystemExit("Google Drive mount failed. Notebook cannot continue. Please check permissions and try again.")




Mounted at /content/drive
Google Drive mounted successfully.


In [ ]:
# --- 2. Define Paths ---
# Base path to your Google Drive content
DRIVE_BASE_PATH = '/content/drive/My Drive/'

# !!! IMPORTANT: SET THIS TO YOUR DATASET'S ROOT FOLDER NAME !!!
DATASET_ROOT_FOLDER_NAME = 'bisindo_dataset_split'

# Full path to your dataset root
DATASET_ROOT_PATH = os.path.join(DRIVE_BASE_PATH, DATASET_ROOT_FOLDER_NAME)

# Name for the main labels directory to be created
LABELS_DIR_NAME = 'labels'
TARGET_LABELS_ROOT_PATH = os.path.join(DATASET_ROOT_PATH, LABELS_DIR_NAME)

# Subdirectories within your dataset that contain images (and need corresponding label folders)
# Adjust this list if your structure is different (e.g., if you don't have a 'test' set)
IMAGE_SETS_TO_PROCESS = ['train', 'val', 'test']

# Supported image extensions (used to identify image files)
IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.webp']

print("--------------------------------------------------")
print(f"Configuration:")
print(f"  Dataset Root Path: {DATASET_ROOT_PATH}")
print(f"  Target Labels Root Path: {TARGET_LABELS_ROOT_PATH}")
print(f"  Image Sets to Process: {IMAGE_SETS_TO_PROCESS}")
print("--------------------------------------------------\n")

--------------------------------------------------
Configuration:
  Dataset Root Path: /content/drive/My Drive/bisindo_dataset_split
  Target Labels Root Path: /content/drive/My Drive/bisindo_dataset_split/labels
  Image Sets to Process: ['train', 'val', 'test']
--------------------------------------------------



In [ ]:
# --- 3. Create Labels Directory Structure ---
print("Starting label directory creation process...\n")

# First, check if the main dataset root path exists
if not os.path.exists(DATASET_ROOT_PATH):
    print(f"ERROR: The specified dataset root path does not exist: {DATASET_ROOT_PATH}")
    print(f"Please ensure the folder '{DATASET_ROOT_FOLDER_NAME}' is correctly named and located directly in your 'My Drive'.")
    print("Script will not proceed.")
else:
    # Create the main 'labels' directory (e.g., .../bisindo_dataset_split/labels/)
    try:
        os.makedirs(TARGET_LABELS_ROOT_PATH, exist_ok=True)
        print(f"Successfully ensured main labels directory exists: {TARGET_LABELS_ROOT_PATH}")
    except OSError as e:
        print(f"ERROR: Could not create main labels directory {TARGET_LABELS_ROOT_PATH}: {e}")
        raise SystemExit("Failed to create main labels directory. Please check permissions.")

    # Process each image set (train, val, test)
    for image_set_name in IMAGE_SETS_TO_PROCESS:
        current_image_set_path = os.path.join(DATASET_ROOT_PATH, image_set_name)
        current_target_label_set_path = os.path.join(TARGET_LABELS_ROOT_PATH, image_set_name)

        print(f"\nProcessing image set: '{image_set_name}'")

        # Check if the image set directory exists (e.g., .../bisindo_dataset_split/train/)
        if not os.path.exists(current_image_set_path):
            print(f"  Warning: Image directory for set '{image_set_name}' not found at {current_image_set_path}. Skipping this set.")
            continue
        else:
            print(f"  Image source directory: {current_image_set_path}")
            print(f"  Target label base directory for this set: {current_target_label_set_path}")

        # Create the base label directory for the current set (e.g., .../labels/train/)
        try:
            os.makedirs(current_target_label_set_path, exist_ok=True)
            print(f"  Successfully ensured label directory for set '{image_set_name}' exists: {current_target_label_set_path}")
        except OSError as e:
            print(f"  ERROR: Could not create label directory {current_target_label_set_path} for set '{image_set_name}': {e}")
            print(f"  Skipping processing for '{image_set_name}' due to this error.")
            continue

        # Walk through the image directory structure for the current set
        # (e.g., .../train/, then .../train/A/, .../train/B/, etc.)
        for dirpath, dirnames, filenames in os.walk(current_image_set_path):
            # dirpath: current directory being scanned (e.g., .../train/A)
            # dirnames: list of subdirectories in dirpath
            # filenames: list of files in dirpath

            # Check if there are any image files in the current directory
            has_images = any(file.lower().endswith(tuple(IMAGE_EXTENSIONS)) for file in filenames)

            if has_images:
                # Determine the relative path of the current image subdirectory
                # from the base of the current image set.
                # Example: if current_image_set_path is .../train/
                # and dirpath is .../train/A/
                # then relative_subdir will be 'A'
                relative_subdir = os.path.relpath(dirpath, current_image_set_path)

                # Construct the full path for the corresponding label subdirectory
                if relative_subdir == '.': # Indicates we are in the root of the image set (e.g. .../train/)
                    target_label_subdir = current_target_label_set_path
                else: # Indicates a subfolder (e.g. .../train/A)
                    target_label_subdir = os.path.join(current_target_label_set_path, relative_subdir)

                # Create this specific label subdirectory (e.g., .../labels/train/A/)
                try:
                    os.makedirs(target_label_subdir, exist_ok=True)
                    print(f"    Ensured label subdirectory exists: {target_label_subdir}")
                except OSError as e:
                    print(f"    ERROR: Could not create label subdirectory {target_label_subdir}: {e}")
            # else:
                # print(f"    No image files found in {dirpath}, skipping label directory creation for it.")


    print("\n--------------------------------------------------")
    print("Label directory creation process finished!")
    print(f"The directory structure for your labels should now be set up under: {TARGET_LABELS_ROOT_PATH}")
    print("Next steps:")
    print("1. Verify the created folder structure in your Google Drive.")
    print("2. Populate these newly created label directories with your '.txt' annotation files.")
    print("   Ensure each image file has a corresponding '.txt' file with the same name (e.g., 'image_A1.jpg' needs 'image_A1.txt').")
    print("--------------------------------------------------")

Starting label directory creation process...

Successfully ensured main labels directory exists: /content/drive/My Drive/bisindo_dataset_split/labels

Processing image set: 'train'
  Image source directory: /content/drive/My Drive/bisindo_dataset_split/train
  Target label base directory for this set: /content/drive/My Drive/bisindo_dataset_split/labels/train
  Successfully ensured label directory for set 'train' exists: /content/drive/My Drive/bisindo_dataset_split/labels/train
    Ensured label subdirectory exists: /content/drive/My Drive/bisindo_dataset_split/labels/train/Q
    Ensured label subdirectory exists: /content/drive/My Drive/bisindo_dataset_split/labels/train/B
    Ensured label subdirectory exists: /content/drive/My Drive/bisindo_dataset_split/labels/train/F
    Ensured label subdirectory exists: /content/drive/My Drive/bisindo_dataset_split/labels/train/A
    Ensured label subdirectory exists: /content/drive/My Drive/bisindo_dataset_split/labels/train/C
    Ensured labe

In [ ]:
print("\n--- Verifying 'labels' directory structure (top levels) ---")
if os.path.exists(TARGET_LABELS_ROOT_PATH):
    print(f"Contents of {TARGET_LABELS_ROOT_PATH}:")
    for item in sorted(os.listdir(TARGET_LABELS_ROOT_PATH)):
        item_path = os.path.join(TARGET_LABELS_ROOT_PATH, item)
        if os.path.isdir(item_path):
            print(f"  - {item}/")
            # List one level deeper for train/val/test
            print(f"    Contents of {item_path}:")
            for sub_item in sorted(os.listdir(item_path)):
                sub_item_path = os.path.join(item_path, sub_item)
                if os.path.isdir(sub_item_path):
                    print(f"      - {sub_item}/")
else:
    print(f"Labels root directory {TARGET_LABELS_ROOT_PATH} was not found for verification.")
print("--- End of verification ---")


--- Verifying 'labels' directory structure (top levels) ---
Contents of /content/drive/My Drive/bisindo_dataset_split/labels:
  - test/
    Contents of /content/drive/My Drive/bisindo_dataset_split/labels/test:
      - A/
      - B/
      - C/
      - D/
      - E/
      - F/
      - G/
      - H/
      - I/
      - J/
      - K/
      - L/
      - M/
      - N/
      - O/
      - P/
      - Q/
      - R/
      - S/
      - T/
      - U/
      - V/
      - W/
      - X/
      - Y/
      - Z/
  - train/
    Contents of /content/drive/My Drive/bisindo_dataset_split/labels/train:
      - A/
      - B/
      - C/
      - D/
      - E/
      - F/
      - G/
      - H/
      - I/
      - J/
      - K/
      - L/
      - M/
      - N/
      - O/
      - P/
      - Q/
      - R/
      - S/
      - T/
      - U/
      - V/
      - W/
      - X/
      - Y/
      - Z/
  - val/
    Contents of /content/drive/My Drive/bisindo_dataset_split/labels/val:
      - A/
      - B/
      - C/
      - D/
  